In [2]:
import matplotlib.pyplot as plt
import numpy as np
import math
from qiskit import IBMQ, Aer, transpile, assemble
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.visualization import plot_histogram
import time

In [33]:
#Create a circuit for Phase estimation for a given eigenvector of a Unitary Operator 

#The circuit maps the phase theta on these qubits, increasing the number of qubits increases the accuracy
num_qubits = 12
circ = QuantumCircuit(num_qubits+1, num_qubits)
circ.x(num_qubits)
for qubit in range(num_qubits):
    circ.h(qubit)

#We want to estimate a gate with θ=1/3
angle = 2*math.pi/3

repetitions = 1
for counting_qubit in range(num_qubits):
    for i in range(repetitions):
        circ.cp(angle, counting_qubit, num_qubits); # This is CU
    repetitions *= 2
#circ.draw('mpl')

In [13]:
#This is a function to apply inverse fourier tranform to circuit passed
def qft_dagger(qc, n):
    for qubit in range(n//2):
        qc.swap(qubit, n-qubit-1)
    for j in range(n):
        for m in range(j):
            qc.cp(-np.pi/float(2**(j-m)), m, j)
        qc.h(j)

In [14]:
circ.barrier()
# Apply inverse QFT
qft_dagger(circ, num_qubits)

A new token can be generated from the blueqat website at https://blueqat.com/accounts/api_setting

In [ ]:
API_KEY = ""
API_ENDPOINT = "https://acceba8a34a7d49bfb20c708e4d7bb12-b00f58c1565b5c4d.elb.us-east-2.amazonaws.com/v2/"

In [15]:
#Sending the Circuit to a Blueqat Server

import gzip
import json
import urllib.request
import time
import ssl
import numpy as np
import collections
from qiskit import QuantumCircuit

shots = 4096
    
def post_request(path, body):
    headers = {
        "Content-Type": "application/json",
        "X-Api-Key": API_KEY,
        "Accept-Encoding": "gzip",
    }
    req = urllib.request.Request(
        API_ENDPOINT + path, json.dumps(body).encode(), headers
    )
    with urllib.request.urlopen(req, context=ctx) as res:
        body = (
            gzip.decompress(res.read())
            if res.headers.get("Content-Encoding") == "gzip"
            else res.read()
    )
    return json.loads(body)
    

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

params = {
    "circuit": circ.qasm(),
    "num_samples": shots,
    "gate_batching": 50,
    "algo": "qr_iteration",
    "cf": 1e-14,
    "chi": 128,
}

# create task
start=time.time()
res = post_request("quantum-tasks/create_itensors_tasks", params)
# get task by ID
#time.sleep(30)


#check if task is completed    
run=True
once=True

while(run):
    time.sleep(1)
    A = post_request("quantum-tasks/get_itensors_task", {'task_id': res["task_id"]})
    #print(A["status"])
    if A["status"]=='failed':
        print('failed')
        run=False
    elif A["status"]=='created' and once==True:
        print('created')
        once=False
    elif A["status"]=='completed':
        print('completed')
        run=False
speed=time.time()-start


# get result
A = post_request("quantum-tasks/get_itensors_task", {'task_id': res["task_id"]})
results=A["result"]

frequency = collections.Counter(results)
print(dict(frequency))
print(speed)

created
completed
{'1010101010101': 2802, '0010101010101': 176, '1110101010101': 113, '0100101010101': 31, '0110101010101': 694, '1101001010101': 4, '1101101010101': 8, '0000101010101': 11, '0001101010101': 45, '0111001010101': 5, '0000011010101': 5, '1100101010101': 50, '1111101010101': 5, '0101101010101': 19, '0011001010101': 6, '1000101010101': 17, '1100011010101': 3, '1001101010101': 23, '0000001010101': 2, '1001011001101': 1, '0111101010101': 3, '1111001010101': 12, '1011001010101': 4, '1011110010101': 2, '1000001010101': 1, '1001001010101': 2, '0101000110101': 1, '0011101010101': 10, '1010001010101': 3, '1011101010101': 5, '0001011010101': 3, '1011011010101': 1, '0100111010101': 1, '1101011010101': 1, '1111101001101': 1, '0011010010101': 1, '0110001010101': 3, '1111011010101': 1, '1001110010101': 1, '1000100110101': 1, '0110101110101': 1, '1001101001001': 1, '1010011010101': 1, '1110011010101': 3, '0001010010101': 1, '0111100010101': 1, '1111110010101': 1, '1011101100101': 1, '11

We get the $|2^n\theta\rangle$ state to be '010101010101', which corresponds to the binary number 1365. To get the value of $\theta$ we divide it by $2^12=4096$ which turns out be 0.333252. This is a very good estimate of the given angle.